# Multiple linear regression

Note that we don't use the variable "credit_checks" in our model. Therefore, the coefficients are different from the example in the lectures.

## Setup

## Data

### Import data

In [ ]:
# ('https://raw.githubusercontent.com/kirenz/datasets/master/loans.csv')

In [ ]:
# select columns
# [['interest_rate', 'verified_income', 'debt_to_income', 'total_credit_utilized', 'public_record_bankrupt', 'term', 'issue_month' ]]

###  Data structure

## Data corrections

In [ ]:
df.___(columns= {'total_credit_utilized' : 'credit_util', 'public_record_bankrupt': 'bankruptcy'}, inplace=True)

Prepare data for scikit-learn model. 

Make dummy variables only for verified_income

In [ ]:
___ = pd.___(___, prefix = ['verified_income', 'issue'])


In [ ]:
# Drop missing values with .dropna (see debt_to_income)


### Variable lists

In [ ]:
# Prepare the data
y_label = "interest_rate"

features = ["verified_income_Source Verified", 
            "verified_income_Verified",
            "debt_to_income",
            "credit_util",
            "bankruptcy",
            "term",
            "issue_Jan-2018",
            "issue_Mar-2018"
            ]



## Model

### Select model

In [ ]:
# Choose the linear regression model


### Fit model

In [ ]:

# Fit the model to the data


### Coefficients

We create a pandas dataframe:

In [ ]:
# Intercept 
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[___]}
    )

intercept

In [ ]:
# Slope coefficients
slope = pd.DataFrame({
    "Name": features,
    "Coefficient": ___}
)

slope

In [ ]:
# combine dataframes with .concat
table = pd.___([intercept, slope], ignore_index=True, sort=False)

round(table, 3)

### Make predictions

In [ ]:
# Make predictions on the data


### Mean squared error